In [1]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
from utils.text import *
from scipy.sparse import find

In [2]:
docs = Doc.objects.filter(query=6956, content__iregex="\w").order_by('id')
docs.count()

232

In [3]:
from nltk.corpus import stopwords as sw
stopwords = set(sw.words('english'))

stopwords | set(["use"])

vec = CountVectorizer(
    ngram_range=(1,2),
    min_df=5, strip_accents='unicode', 
    max_features=10000,
    stop_words=stopwords,
    tokenizer=snowball_stemmer()
)

#X = vec.fit_transform(docs.values_list('content',flat=True))
def mash_texts(x):
    s = ""
    for at in ["de", "kwp"]:#,"ab", "ti",]:
        if hasattr(x.wosarticle, at) and getattr(x.wosarticle, at):
            s+=f" {getattr(x.wosarticle, at)}"
    return s.strip()
texts = [mash_texts(x) for x in docs]
X = X = vec.fit_transform(texts)
vocab = vec.get_feature_names()
X[X.nonzero()] = 1
X

<232x177 sparse matrix of type '<class 'numpy.int64'>'
	with 2034 stored elements in Compressed Sparse Row format>

In [4]:
print([v for v in vocab if " " in v])

['chang climat', 'climat chang', 'climat variabl', 'climatechang massbal', 'coral reef', 'global chang', 'global warm', 'ice cap', 'mountain glacier', 'rang shift', 'regim shift', 'remot sens', 'respons impact', 'sealevel rise', 'seasurfac temperatur']


In [5]:
indices, vals = np.array(X.sum(0))[0].argsort()[::-1], np.sort(np.array(X.sum(0))[0])[::-1]

In [6]:
texts[0]

'alpine plants; Argyroxyphium sandwicense; biodiversity loss; climate change ecology; hotspot; population declines; silversword SILVERSWORD ALLIANCE COMPOSITAE; INDUCED TREE MORTALITY; GIANT ROSETTE PLANT; DROUGHT; ALPINE; TEMPERATURE; EXTINCTION; MOUNTAIN; MADIINAE; FOREST'

In [7]:
# get the combination of relevant words that return all documents
exclusions = []
stop = [
    "increas","use","data","studi","result","global","variabl","effect","show","model","current",
    "year","respons","also","observ","relat","level","signific","global warm","trend","growth",
    "simul", "sea", "ecosystem", "marin","forest","product","water"
]

for j in range(1):
    ids = []
    for i in range(20):
        if i==0:
            for ind in indices:
                w = vocab[ind]
                if w in stop:
                    continue
                if ind not in exclusions and ind not in ids:
                    x = X[:,ind]
                    ids.append(ind)
                    exclusions.append(ind)
                    break
            continue
        max_x = len(x.data)
        max_ind = None
        for ind in indices:
            w = vocab[ind]
            if w in stop:
                continue
            if ind in ids or ind in exclusions:
                continue
            tmp_x = x + X[:,ind]
            if len(tmp_x.data) > max_x:
                max_x = len(tmp_x.data)
                max_ind = ind
        if not max_ind:
            break
        else:
            x = x + X[:,max_ind]
            ids.append(max_ind)

    strings = [f'"{vocab[x]}*"' for x in ids]
    print(f'{" OR ".join(strings)} returns {len(x.data)}')

"climat*" OR "climatechang*" OR "temperatur*" OR "impact*" OR "shift*" OR "chang*" OR "drought*" OR "island*" OR "veget*" OR "glacier*" OR "conserv*" OR "massbal*" OR "popul*" OR "ecolog*" OR "sens*" OR "biodivers*" OR "rang*" OR "precipit*" OR "river*" OR "mountain*" returns 214


In [8]:
x = X[:,1]+X[:,2]
#l = lx[x > 1].shape[1]

x[x==1] = 0
x.eliminate_zeros()
x[x>0] = 1
x.getnnz()

0

In [9]:
kws = [vocab[x] for x in ids]
print(kws)

['climat', 'climatechang', 'temperatur', 'impact', 'shift', 'chang', 'drought', 'island', 'veget', 'glacier', 'conserv', 'massbal', 'popul', 'ecolog', 'sens', 'biodivers', 'rang', 'precipit', 'river', 'mountain']


In [10]:
#X = vec.fit_transform(docs.values_list('content',flat=True))

vec = CountVectorizer(
    ngram_range=(1,2),
    min_df=10, strip_accents='unicode', 
    max_features=10000,
    stop_words=stopwords,
    tokenizer=snowball_stemmer()
)

def mash_texts(x):
    s = ""
    for at in ["de", "kwp","ab", "ti",]:
        if hasattr(x.wosarticle, at) and getattr(x.wosarticle, at):
            s+=f" {getattr(x.wosarticle, at)}"
    return s.strip()
texts = [mash_texts(x) for x in docs]
X = X = vec.fit_transform(texts)
vocab = vec.get_feature_names()
X[X.nonzero()] = 1
X
    


<232x618 sparse matrix of type '<class 'numpy.int64'>'
	with 16082 stored elements in Compressed Sparse Row format>

## Only Climate keywords

In [11]:
# get the combination of relevant words that return all documents
indices, vals = np.array(X.sum(0))[0].argsort()[::-1], np.sort(np.array(X.sum(0))[0])[::-1]

exclusions = []
stop = [
    "increas","use","data","studi","result","global","variabl","effect","show","model","current",
    "year","respons","also","observ","relat","level","signific","trend","growth",
    "simul", "sea", "ecosystem", "marin","forest","product","water",'alter','alp','although','consist',
    'amount','chang','area','dur','dynam','becom'
]

for j in range(1):
    ids = []
    for i in range(20):
        if i==0:
            for ind in indices:
                w = vocab[ind]
                if w in stop:
                    continue
                if ind not in exclusions and ind not in ids:
                    x = X[:,ind]
                    ids.append(ind)
                    exclusions.append(ind)
                    break
            continue
        max_x = len(x.data)
        max_ind = None
        for ind in indices:
            w = vocab[ind]
            if w in stop:
                continue
            if ind in ids or ind in exclusions:
                continue
            tmp_x = x + X[:,ind]
            if len(tmp_x.data) > max_x:
                max_x = len(tmp_x.data)
                max_ind = ind
        if not max_ind:
            break
        else:
            x = x + X[:,max_ind]
            ids.append(max_ind)

    strings = [f'"{vocab[x]}*"' for x in ids]
    print(f'{" OR ".join(strings)} returns {len(x.data)}')

"climat*" OR "region*" OR "impact*" OR "occur*" OR "caus*" OR "may*" OR "temperatur*" returns 232


In [88]:
pats = [
    "climate model",
    "elevated\w* temperatur",
    "ocean\w* warming",
    "saline\w* intrusion",
    "chang\w* climat",
    "environment\w* change",
    "climat\w* change,",
    "climat\w* warm",
    "warming\w* climat",
    "climat\w* varia",
    "global\w* warming",
    "greenhouse\w* effect",
    "anthropogen\w*",
    "sea\w* level",
    "precipitation variabil",
    "temperature\w* impact",
    "environmental\w* variab",
    "change\w* (\w+\s*\W*){5} cryosphere",
    "weather\w* pattern",
    "weather\w* factor\w*",
]
rpat = "|".join([x.replace(" ","( |-)") for x in pats])

#" OR ".join([x.replace('\w*','*').replace('(\w+\s*\W*){','NEAR/').replace('}','') for x in pats])
opats = " OR ".join([x.replace('\w*','*') for x in pats if '(\w+\s*\W*){' not in x])
npats = " OR ".join([x.replace('\w*','*').replace('(\w+\s*\W*){','NEAR/').replace('}',' ') for x in pats if '(\w+\s*\W*){' in x])
    
wpat = f"TS=({opats}) OR TS=({npats})"

print(wpat)

#for s in pats:
    #if 
# Climate in keywords

## WHY IS OLIVEIRA, 2007 included?
### Also, Nepstad and Stickler, 2008
### Polidoro 2010, very weak - not main conclusion

# Exclude non-climate docs
docs = docs.exclude(pk__in=[
    1622783, # Why is Oliveira, 2007 included?
    1627701, # Also, Nepstad and Stickler, 2008
    2090005, # Nepstad, 2006
    2089960, # Wassenaar, 2007 - LUC
    1627733, # Polidoro 2010, very weak - not main conclusion
    2090004, # Veran 2007 - long line fishing
    2089967, # Giri - mangrove database, climate only mentioned in conclusion and with reference
    2089944, # Jellyman - eels
    2091676, # Novelo-Casanova, 2010: Cayman Islands climate??
    2089959, # Bruno, 2007 
    2091680, # Guzman, 2008, we consider the main anthropogenic threats to the coral reefs and communities of the islands are overfishing, sedimentation and tourism
    2089950, # Razumov - permafrost and erosion - climate? is the ref there as counter-evidence Nevertheless, the longterm average annual rate
#                             of erosion of the coast line of the bottom of the Arctic
#                             Seas is much lower than in the seas outside the cry
#                             olithozone even under the conditions of the climate’s
#                             warming accompanied by the activation of thermo
#                             abrasion and during the subaqual abrasion processes
#                             (Fig. 5). In particular, this proves the development of
#                             ordinary abrasionaccumulation processes in the Arc
#                             tic Seas, and, here, we cannot say about the influence
#                             of thermal slumps on the formation of the underwater
#                             slope relief
])

#

print(f"starting point: {docs.count()} docs")



tmatch = docs.filter(title__iregex=rpat).values_list('pk',flat=True)
abmatch = docs.filter(content__iregex=rpat).values_list('pk',flat=True)
dematch = docs.filter(wosarticle__de__iregex=rpat).values_list('pk',flat=True)
kwpmatch = docs.filter(wosarticle__kwp__iregex=rpat).values_list('pk',flat=True)

declim = docs.filter(wosarticle__de__icontains="climat").values_list('pk',flat=True)
kwpclim = docs.filter(wosarticle__kwp__icontains="climat").values_list('pk', flat=True)
ticlim = docs.filter(title__icontains="climat").values_list('pk', flat=True)

mids = set(tmatch) | set(abmatch) | set(dematch) | set(kwpmatch) | set(declim) | set(kwpclim) | set(ticlim)

mdocs = docs.filter(pk__in=mids)

print(mdocs.count())

nmdocs=docs.exclude(pk__in=mids)

for d in nmdocs[:10]:
    print('\n#####')
    print(d.id, d.title)
    print(d.docauthinst_set.all().order_by('position'))
    print(d.PY)
    print(d.content)
    print(d.wosarticle.de)
    print(d.wosarticle.kwp)

TS=(climate model OR elevated* temperatur OR ocean* warming OR saline* intrusion OR chang* climat OR environment* change OR climat* change, OR climat* warming OR warming* climat OR climat* varia OR global* warming OR greenhouse* effect OR anthropogen* OR sea* level OR precipitation variabil OR temperature* impact OR environmental* variab OR weather* pattern OR weather* factor*) OR TS=(change* NEAR/5  cryosphere)
starting point: 220 docs
217

#####
1461495 The Major Floods in the Amazonas River and Tributaries (Western Amazon Basin) during the 1970-2012 Period: A Focus on the 2012 Flood
<QuerySet [<DocAuthInst: Espinoza, JC>, <DocAuthInst: Espinoza, JC>, <DocAuthInst: Ronchail, J>, <DocAuthInst: Ronchail, J>, <DocAuthInst: Frappart, F>, <DocAuthInst: Frappart, F>, <DocAuthInst: Lavado, W>, <DocAuthInst: Lavado, W>, <DocAuthInst: Santini, W>, <DocAuthInst: Santini, W>, <DocAuthInst: Guyot, JL>, <DocAuthInst: Guyot, JL>]>
2013
In this work, the authors analyze the origin of the extreme fl

In [89]:
from utils.utils import flatten

climate_words = list(flatten([[y.replace('\w*','') for y in x.split() if re.match('\w{2,}',y)] for x in pats]))

climate_words

['climate',
 'model',
 'elevated',
 'temperatur',
 'ocean',
 'warming',
 'saline',
 'intrusion',
 'chang',
 'climat',
 'environment',
 'change',
 'climat',
 'change,',
 'climat',
 'warming',
 'warming',
 'climat',
 'climat',
 'varia',
 'global',
 'warming',
 'greenhouse',
 'effect',
 'anthropogen',
 'sea',
 'level',
 'precipitation',
 'variabil',
 'temperature',
 'impact',
 'environmental',
 'variab',
 'change',
 'cryosphere',
 'weather',
 'pattern',
 'weather',
 'factor']

In [90]:
texts = [mash_texts(x) for x in mdocs]
X = X = vec.fit_transform(texts)
vocab = vec.get_feature_names()
X[X.nonzero()] = 1

In [91]:
# Find combinations of vocab (except for those that contain the stopwords below)

indices, vals = np.array(X.sum(0))[0].argsort()[::-1], np.sort(np.array(X.sum(0))[0])[::-1]

all_cs = list(combinations(indices[:300],2))

print(len(all_cs))

c_lengths = []
cs = []
max_x = 0
max_ind = 0
local_stops = set([
    "use","also","studi","may","larg","dure","occur","result",
    "climat chang","year"
]) | set(words)

stop_combos = [
    ["studi","year"],
    ["year","data"],
    ["year","observ"],
    ["year","dure"],
    ["data","indic"],
    ["water","ocean"],
    ["increase", "result"],
    ["sea","ocean"],
    ["chang", "increas"],
    ["increas", "temperatur"],
    ["data", "analysi"],
    ["impact","effect"],
    ["result", "high"],
    ["chang","warm"],
    ["climat","warm"],
    ["climat","temperatur"],
    ["show", "data"]
]

for c in all_cs:
    words = [vocab[x] for x in c]
    if len(local_stops & set(words))>0:
        continue
    if words in stop_combos:
        continue
    if len(set(words) & set(climate_words)) > 0:
        continue
    x = sum([X[:,x] for x in c])
    l = x[x > 1].shape[1]
    c_lengths.append(l)   
    cs.append(c)
    
print(len(cs))

44850
37125


In [92]:
ids = []

# Go through the combinations, choosing the one that adds the most relevant documents
seen = []
for i in range(25):
    
    if i==0:
        c = cs[np.argsort(c_lengths)[::-1][0]]
        x = sum([X[:,xi] for xi in c])
        x[x==1] = 0
        x.eliminate_zeros()
        x[x>0] = 1
        ids.append(c)
        continue

    base_l = len(x.data)
    max_l = len(x.data)
    max_ind = None
    
    for ind in np.argsort(c_lengths)[::-1]:
        if ind in seen:
            continue
        c = cs[ind]
        words = [vocab[x] for x in c]
        if len(local_stops & set(words))>0:
            continue
        if words in stop_combos:
            continue
        if c in ids:
            continue
        tmp_x = sum([X[:,xi] for xi in c])
        for w in range(1, len(c)):
            tmp_x[tmp_x==w] = 0
        tmp_x.eliminate_zeros()
        tmp_x[tmp_x>0] = 1
        tmp_x = x + tmp_x
        l = len(tmp_x.data)
        if l > max_l:
            max_l = l
            max_ind = ind
        if l <= base_l: 
            seen.append(ind)
            
    if not max_ind:
        break
    else:
        c = cs[max_ind]
        tmp_x = sum([X[:,xi] for xi in c])
        for w in range(1, len(c)):
            tmp_x[tmp_x==w] = 0
        tmp_x.eliminate_zeros()
        tmp_x[tmp_x>0] = 1
        x = x + tmp_x
        x[x>1]=1
        ids.append(c)
        print(f"{i+1}: {max_l}")
        if x.shape[0] == max_l:
            break
                
c = [" AND ".join([vocab[x]+"*" for x in c]) for c in ids]
print(c)
print(max_l)       


2: 101
3: 127
4: 144
5: 157
6: 169
7: 179
8: 188
9: 194
10: 199
11: 203
12: 207
13: 210
14: 213
15: 215
16: 217
['increas* AND region*', 'warm* AND respons*', 'increas* AND trend*', 'period* AND area*', 'show* AND relat*', 'speci* AND ecosystem*', 'increas* AND data*', 'variabl* AND mean*', 'water* AND surfac*', 'warm* AND associ*', 'region* AND data*', 'speci* AND shift*', 'increas* AND scale*', 'ice* AND glacier*', 'signific* AND time*', 'rate* AND develop*']
217


In [64]:
t = " OR ".join([f"({x})" for x in c])
q = f"TS=({t})"
print(q)

TS=((climat* AND temperatur*) OR (chang* AND impact*) OR (warm* AND global*) OR (chang* AND water*) OR (climat* AND adapt*) OR (increas* AND temperatur*) OR (climat* AND sea*) OR (climat* AND rainfal*) OR (chang* AND forest*) OR (impact* AND year*) OR (climat* AND precipit*) OR (climatechang* AND ecosystem*) OR (impact* AND variabl*) OR (chang* AND ecolog*) OR (water* AND sea*) OR (climat chang* AND warm*) OR (impact* AND data*) OR (extrem* AND rainfal*) OR (chang* AND ecosystem*) OR (chang* AND communiti*) OR (region* AND water*) OR (increas* AND impact*) OR (increas* AND warm*) OR (rang* AND ecolog*))


In [ ]:
X.shape

In [14]:
texts[np.where(x.A==0)[0][0]]

"Background. A number of factors have recently caused mass coral mortality events in all of the world's tropical oceans. However, little is known about the timing, rate or spatial variability of the loss of reef-building corals, especially in the Indo-Pacific, which contains 75% of the world's coral reefs. Methodology/Principle Findings. We compiled and analyzed a coral cover database of 6001 quantitative surveys of 2667 Indo-Pacific coral reefs performed between 1968 and 2004. Surveys conducted during 2003 indicated that coral cover averaged only 22.1% (95% CI: 20.7, 23.4) and just 7 of 390 reefs surveyed that year had coral cover > 60%. Estimated yearly coral cover loss based on annually pooled survey data was approximately 1% over the last twenty years and 2% between 1997 and 2003 (or 3,168 km(2) per year). The annual loss based on repeated measures regression analysis of a subset of reefs that were monitored for multiple years from 1997 to 2004 was 0.72% (n = 476 reefs, 95% CI: 0.3

In [15]:
np.where(x.A==0)

for j in list(find(X[353]))[1]:
    print(vocab[j])

IndexError: index (353) out of range

In [16]:
# Climate words

cwords = ["climate change","climat"]
for t in texts:
    tc = False
    for w in cwords:
        if w in t.lower():
            tc=True
    if not tc:
        print(t)
        break


Effective conservation requires rigorous baselines of pristine conditions to assess the impacts of human activities and to evaluate the efficacy of management. Most coral reefs are moderately to severely degraded by local human activities such as fishing and pollution as well as global change, hence it is difficult to separate local from global effects. To this end, we surveyed coral reefs on uninhabited atolls in the northern Line Islands to provide a baseline of reef community structure, and on increasingly populated atolls to document changes associated with human activities. We found that top predators and reef-building organisms dominated unpopulated Kingman and Palmyra, while small planktivorous fishes and fleshy algae dominated the populated atolls of Tabuaeran and Kiritimati. Sharks and other top predators overwhelmed the fish assemblages on Kingman and Palmyra so that the biomass pyramid was inverted (top-heavy). In contrast, the biomass pyramid at Tabuaeran and Kiritimati exh

In [17]:
# for i,c in enumerate(combinations(indices, 4)):
#     x = sum([X[:,x] for x in c])
#     l = len(x.data)
#     if l > 740:
#         print(f'{" OR ".join([vocab[x] for x in c])} returns {l}')

In [18]:
ids = []
for i in range(10):
    if i==0:
        x = X[:,indices[0]]
        ids.append(indices[0])
        continue
    max_x = len(x.data)
    max_ind = None
    for ind in indices:
        if ind in ids:
            continue
        tmp_x = x + X[:,ind]
        if len(tmp_x.data) > max_x:
            max_x = len(tmp_x.data)
            max_ind = ind
    if not max_ind:
        break
    else:
        x = x + X[:,max_ind]
        ids.append(max_ind)
        print(max_x)
        
print(i)
print(f'{" OR ".join([vocab[x]+"*" for x in ids])} returns {len(x.data)}')

209
218
225
228
229
6
chang* OR region* OR area* OR impact* OR sea* OR climat* returns 229


In [19]:
n_ids = []
for i in range(10):
    if i==0:
        for ind in indices:
            if ind not in ids:
                x = X[:,ind]
                n_ids.append(ind)
                break
        continue
    max_x = len(x.data)
    max_ind = None
    for ind in indices:
        if ind in ids or ind in n_ids:
            continue
        tmp_x = x + X[:,ind]
        if len(tmp_x.data) > max_x:
            max_x = len(tmp_x.data)
            max_ind = ind
    if not max_ind:
        break
    else:
        x = x + X[:,max_ind]
        n_ids.append(max_ind)
        print(max_x)
        
print(i)
strings = [f'"{vocab[x]}*"' for x in n_ids]
print(f'{" OR ".join(strings)} returns {len(x.data)}')

181
204
217
223
226
228
229
8
"increas*" OR "use*" OR "climat chang*" OR "year*" OR "show*" OR "studi*" OR "may*" OR "time*" returns 229


In [20]:
ids += n_ids
n_ids = []
for i in range(10):
    if i==0:
        for ind in indices:
            if ind not in ids:
                x = X[:,ind]
                n_ids.append(ind)
                break
        continue
    max_x = len(x.data)
    max_ind = None
    for ind in indices:
        if ind in ids or ind in n_ids:
            continue
        tmp_x = x + X[:,ind]
        if len(tmp_x.data) > max_x:
            max_x = len(tmp_x.data)
            max_ind = ind
    if not max_ind:
        break
    else:
        x = x + X[:,max_ind]
        n_ids.append(max_ind)
        print(max_x)
        
print(i)
strings = [f'"{vocab[x]}*"' for x in n_ids]
print(f'{" OR ".join(strings)} returns {len(x.data)}')

157
186
203
213
220
224
227
229
9
"temperatur*" OR "relat*" OR "model*" OR "speci*" OR "result*" OR "period*" OR "effect*" OR "reserv*" OR "condit*" returns 229


In [21]:
ids += n_ids
n_ids = []
for i in range(10):
    if i==0:
        for ind in indices:
            if ind not in ids:
                x = X[:,ind]
                n_ids.append(ind)
                break
        continue
    max_x = len(x.data)
    max_ind = None
    for ind in indices:
        if ind in ids or ind in n_ids:
            continue
        tmp_x = x + X[:,ind]
        if len(tmp_x.data) > max_x:
            max_x = len(tmp_x.data)
            max_ind = ind
    if not max_ind:
        break
    else:
        x = x + X[:,max_ind]
        n_ids.append(max_ind)
        print(max_x)
        
print(i)
print(f'{" OR ".join([vocab[x] for x in n_ids])} returns {len(x.data)}')

150
179
199
210
217
222
226
228
229
9
warm OR trend OR data OR high OR present OR water OR event OR dynam OR also OR global returns 229


In [22]:
ids += n_ids
n_ids = []
for i in range(10):
    if i==0:
        for ind in indices:
            if ind not in ids:
                x = X[:,ind]
                n_ids.append(ind)
                break
        continue
    max_x = len(x.data)
    max_ind = None
    for ind in indices:
        if ind in ids or ind in n_ids:
            continue
        tmp_x = x + X[:,ind]
        if len(tmp_x.data) > max_x:
            max_x = len(tmp_x.data)
            max_ind = ind
    if not max_ind:
        break
    else:
        x = x + X[:,max_ind]
        n_ids.append(max_ind)
        print(max_x)
        
print(i)
print(f'{" OR ".join([vocab[x] for x in n_ids])} returns {len(x.data)}')

140
169
187
201
212
221
225
228
229
9
respons OR signific OR observ OR rate OR variabl OR recent OR provid OR differ OR sever OR shift returns 229
